In [177]:
import pandas as pd
import numpy as np
import csv 
import os 
import re

pd.set_option("display.max_rows",101)
pd.set_option("display.max_columns",101)
pd.set_option("display.max_seq_items",500)

In [246]:
def read_csv(csv_file):
    '''
    '''
    assert os.path.isfile(csv_file), "csv file does not exist"

    # convert the pd cols from NHGIS names
    data = pd.read_csv(csv_file, low_memory=False)

    # only keep IL
    working_df = data[(data.STATE == 'Illinois') & (data.COUNTY == 'Cook')]
    
    # SET INDEX
    working_df = working_df.set_index('GISJOIN')
    
    return working_df

In [260]:
df97 = read_csv('./nhgis0016_ds97_1970_tract.csv')
df97_2 = read_csv('./nhgis0017_ds97_1970_tract.csv')

ALL_PEOPLE = df97['CY7001'] # table 97
BLACK_PEOPLE = df97['CY8001'] # table 97
OCCUPIED = df97['CU9001'] # table 97
VACANT = df97['CU9002']+ df97['CU9003']+df97['CU9004'] +\
    df97['CU9005'] + df97['CU9006']
TOTAL_UNITS = df97_2.CZQ001

    
    
df96 = read_csv('./nhgis0009_ds96_1970_tract.csv')

NON_WHITE_PEOPLE = df96.CM6003 + df96.CM6004
OWNER = df96.CK3001 + df96.CK4001 # white and black are separately tabulated
RENTER = df96.CK4002 + df96.CK3002 



In [268]:
final_df = df97[['YEAR', 'TRACTA', 'COUNTY', 'STATE']]
final_df = final_df.assign(total_pop = ALL_PEOPLE)
final_df = final_df.assign(Pwhite = (ALL_PEOPLE - BLACK_PEOPLE - 
                          NON_WHITE_PEOPLE)/ALL_PEOPLE)
final_df = final_df.assign(Pblack = BLACK_PEOPLE/ALL_PEOPLE)
final_df = final_df.assign(Pnonwhite = NON_WHITE_PEOPLE/ALL_PEOPLE)
final_df = final_df.assign(units = TOTAL_UNITS)
final_df = final_df.assign(occupied = OCCUPIED/TOTAL_UNITS)
final_df = final_df.assign(vacant = VACANT/TOTAL_UNITS)

########## HACK HACK HACK

final_df = final_df.assign(own = OWNER/(OWNER+RENTER))
final_df = final_df.assign(rent = RENTER/(OWNER+RENTER))
final_df = final_df.reset_index()

In [269]:
column_names = ['GISJOIN','YEAR', 'TRACTA', 'COUNTY','STATE', 'Total Pop', '% White', 
               '% Black', '% Other races', 'Total Units', '% Occupied', 
               '% Vacant', '% Owner Occupied', '% Renter']

final_df.to_csv(path_or_buf='1970S_fordb.csv', index=False, header=column_names)

In [270]:
final_df.head(10)

,GISJOIN,YEAR,TRACTA,COUNTY,STATE,total_pop,Pwhite,Pblack,Pnonwhite,units,occupied,vacant,own,rent
0,G17003100101,1970,101,Cook,Illinois,6969,0.911178,0.055819,0.033003,3524,0.952327,0.047673,0.047692,0.952308
1,G17003100102,1970,102,Cook,Illinois,8771,0.958728,0.003990,0.037282,3908,0.965455,0.034545,0.130150,0.869850
2,G17003100103,1970,103,Cook,Illinois,6744,0.958037,0.010824,0.031139,3318,0.955093,0.044907,0.111899,0.888101
3,G17003100104,1970,104,Cook,Illinois,5584,0.969914,0.016476,0.013610,2416,0.973924,0.026076,0.096593,0.903407
4,G17003100105,1970,105,Cook,Illinois,11118,0.953769,0.009534,0.036697,5516,0.961929,0.038071,0.055749,0.944251
5,G17003100106,1970,106,Cook,Illinois,7181,0.969085,0.002228,0.028687,3265,0.950689,0.049311,0.102437,0.897563
6,G17003100107,1970,107,Cook,Illinois,8848,0.961573,0.003843,0.034584,3861,0.961150,0.038850,0.149225,0.850775
7,G17003100108,1970,108,Cook,Illinois,4037,0.971761,0.000743,0.027496,1744,0.966743,0.033257,0.135870,0.864130
8,G17003100109,1970,109,Cook,Illinois,1529,0.924133,0.009810,0.066056,479,0.977035,0.022965,0.200893,0.799107
9,G17003100201,1970,201,Cook,Illinois,3606,0.992790,0.002773,0.004437,1497,0.974616,0.025384,0.276934,0.723066
